# Quantitative Momentum Investment Strategy
#### The program selects the top 50 companies whose price has steadily increased the most, then outputs a list of recommended trades for an equal-weight portfolio. A basic implementation and a more sophisticated version are provided. The second version outputs an Excel file with recommended trades.

In [65]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
from statistics import mean
import xlsxwriter
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
stocks = pd.read_csv('CSV_files/sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [67]:
IEX_CLOUD_API_TOKEN = 'pk_682c76a7979e43349b214ad64d6d41b7'

In [68]:
symbol='AAPL'
api_url = f'https://arbitrage.iex.cloud/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2728017141680,
 'week52high': 197.7,
 'week52low': 123.48,
 'week52highSplitAdjustOnly': 198.23,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.026026099601385244,
 'sharesOutstanding': 15634232000,
 'float': 0,
 'avg10Volume': 52299622,
 'avg30Volume': 56135299,
 'day200MovingAvg': 170.94,
 'day50MovingAvg': 187.35,
 'employees': 164000,
 'ttmEPS': 5.95,
 'ttmDividendRate': 0.936842139930527,
 'dividendYield': 0.005369030545764954,
 'nextDividendDate': '',
 'exDividendDate': '2023-08-11',
 'nextEarningsDate': '2023-11-02',
 'peRatio': 28.78869925791473,
 'beta': 1.290902798699239,
 'maxChangePercent': 67.0219865897396,
 'year5ChangePercent': 2.391197540317722,
 'year2ChangePercent': 0.1942110637505714,
 'year1ChangePercent': 0.026026099601385244,
 'ytdChangePercent': 0.3505041655186081,
 'month6ChangePercent': 0.1484992631411972,
 'month3ChangePercent': -0.0011328775490461185,
 'month1ChangePercent': -0.09407376064005146,
 'day30Chan

In [69]:
def chunks(lst, n):
    """
    Yield successive n-sized chunks from lst.
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [70]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range (0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
for symbol_string in symbol_strings:
    print(symbol_string)

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS
GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INF

## Basic Program:
#### Companies are picked solely by One-Year Price Return (both low- and high-quality momentum stocks).

In [71]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,118.325,-0.12761,N/A
1,AAL,15.040,0.061397,N/A
2,AAP,67.755,-0.650552,N/A
3,AAPL,175.400,0.026026,N/A
4,ABBV,149.660,0.101382,N/A
...,...,...,...,...
496,YUM,128.840,0.12264,N/A
497,ZBH,119.800,0.060681,N/A
498,ZBRA,268.155,-0.171828,N/A
499,ZION,34.225,-0.369595,N/A


In [72]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.920,2.281287,N/A
1,388,RCL,98.490,1.501892,N/A
2,343,NVDA,461.070,1.428107,N/A
3,193,FTI,18.055,1.15703,N/A
4,368,PHM,77.620,0.897579,N/A
5,197,GE,111.610,0.834728,N/A
6,281,LLY,549.170,0.711245,N/A
7,330,NFLX,405.870,0.677434,N/A
8,132,DHI,115.560,0.594798,N/A
9,410,SLB,57.235,0.584645,N/A


In [73]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('Please enter an integer.')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()

Enter the size of your portfolio: 10000000


In [ ]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe

## Improved Version:
#### Only high-quality momentum stocks are taken into consideration, based on the highest percentiles of 1-month, 3-month, 6-month, and 1-year returns.

In [114]:
hqm_columns = [
    'Ticker', 
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            '0',
            data[symbol]['stats']['year1ChangePercent'],
            '0',
            data[symbol]['stats']['month6ChangePercent'],
            '0',
            data[symbol]['stats']['month3ChangePercent'],
            '0',
            data[symbol]['stats']['month1ChangePercent'],
            '0',
            '0'
        ],  
        index = hqm_columns),
        ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.060,0,-0.12761,0,-0.192821,0,-0.072882,0,-0.042553,0,0
1,AAL,15.045,0,0.061397,0,-0.080685,0,0.014845,0,-0.137615,0,0
2,AAP,67.730,0,-0.650552,0,-0.526937,0,-0.403137,0,-0.002571,0,0
3,AAPL,175.315,0,0.026026,0,0.148499,0,-0.001133,0,-0.094074,0,0
4,ABBV,149.810,0,0.101382,0,0.012497,0,0.046134,0,0.055911,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,128.890,0,0.12264,0,-0.017551,0,-0.061886,0,-0.04975,0,0
497,ZBH,119.995,0,0.060681,0,-0.05052,0,-0.115717,0,-0.164997,0,0
498,ZBRA,268.340,0,-0.171828,0,-0.15686,0,-0.03141,0,-0.106915,0,0
499,ZION,34.270,0,-0.369595,0,-0.30488,0,0.312408,0,-0.073004,0,0


In [116]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.060,0,-0.12761,0.251497,-0.192821,0.105788,-0.072882,0.135729,-0.042553,0.493014,0
1,AAL,15.045,0,0.061397,0.616766,-0.080685,0.287425,0.014845,0.46507,-0.137615,0.077844,0
2,AAP,67.730,0,-0.650552,0.003992,-0.526937,0.005988,-0.403137,0.003992,-0.002571,0.676647,0
3,AAPL,175.315,0,0.026026,0.546906,0.148499,0.864271,-0.001133,0.367265,-0.094074,0.177645,0
4,ABBV,149.810,0,0.101382,0.700599,0.012497,0.580838,0.046134,0.580838,0.055911,0.894212,0
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,128.890,0,0.12264,0.736527,-0.017551,0.487026,-0.061886,0.161677,-0.04975,0.443114,0
497,ZBH,119.995,0,0.060681,0.61477,-0.05052,0.393214,-0.115717,0.083832,-0.164997,0.03992,0
498,ZBRA,268.340,0,-0.171828,0.169661,-0.15686,0.143713,-0.03141,0.255489,-0.106915,0.133733,0
499,ZION,34.270,0,-0.369595,0.03992,-0.30488,0.023952,0.312408,0.966068,-0.073004,0.293413,0


In [117]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.060,0,-0.12761,0.251497,-0.192821,0.105788,-0.072882,0.135729,-0.042553,0.493014,0.246507
1,AAL,15.045,0,0.061397,0.616766,-0.080685,0.287425,0.014845,0.46507,-0.137615,0.077844,0.361776
2,AAP,67.730,0,-0.650552,0.003992,-0.526937,0.005988,-0.403137,0.003992,-0.002571,0.676647,0.172655
3,AAPL,175.315,0,0.026026,0.546906,0.148499,0.864271,-0.001133,0.367265,-0.094074,0.177645,0.489022
4,ABBV,149.810,0,0.101382,0.700599,0.012497,0.580838,0.046134,0.580838,0.055911,0.894212,0.689122
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,128.890,0,0.12264,0.736527,-0.017551,0.487026,-0.061886,0.161677,-0.04975,0.443114,0.457086
497,ZBH,119.995,0,0.060681,0.61477,-0.05052,0.393214,-0.115717,0.083832,-0.164997,0.03992,0.282934
498,ZBRA,268.340,0,-0.171828,0.169661,-0.15686,0.143713,-0.03141,0.255489,-0.106915,0.133733,0.175649
499,ZION,34.270,0,-0.369595,0.03992,-0.30488,0.023952,0.312408,0.966068,-0.073004,0.293413,0.330838


In [118]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LLY,549.280,0,0.711245,0.988024,0.67182,0.996008,0.237853,0.938124,0.180633,0.996008,0.979541
1,LB,79.920,0,2.281287,1.0,0.815831,0.998004,0.215421,0.918164,0.079271,0.94012,0.964072
2,MPC,142.010,0,0.474625,0.966068,0.194662,0.916168,0.317686,0.972056,0.169851,0.994012,0.962076
3,FTI,18.110,0,1.15703,0.994012,0.304065,0.96008,0.316284,0.97006,0.045792,0.866267,0.947605
4,COG,22.250,0,0.340483,0.92016,0.180873,0.896208,0.255629,0.952096,0.249298,0.998004,0.941617
5,ANET,183.370,0,0.388769,0.938124,0.306084,0.964072,0.254534,0.9501,0.053264,0.886228,0.934631
6,APA,43.810,0,0.26199,0.866267,0.183977,0.902196,0.341126,0.974052,0.164604,0.992016,0.933633
7,PSX,112.970,0,0.343237,0.922156,0.17119,0.886228,0.214545,0.916168,0.108206,0.972056,0.924152
8,PHM,77.330,0,0.897579,0.992016,0.467768,0.994012,0.148771,0.852295,0.022962,0.816367,0.913673
9,HES,153.280,0,0.350265,0.924152,0.15534,0.874251,0.168551,0.88024,0.108451,0.974052,0.913174


In [85]:
portfolio_input()

Enter the size of your portfolio: 10000000


In [119]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LLY,549.280,364,0.711245,0.988024,0.67182,0.996008,0.237853,0.938124,0.180633,0.996008,0.979541
1,LB,79.920,2502,2.281287,1.0,0.815831,0.998004,0.215421,0.918164,0.079271,0.94012,0.964072
2,MPC,142.010,1408,0.474625,0.966068,0.194662,0.916168,0.317686,0.972056,0.169851,0.994012,0.962076
3,FTI,18.110,11043,1.15703,0.994012,0.304065,0.96008,0.316284,0.97006,0.045792,0.866267,0.947605
4,COG,22.250,8988,0.340483,0.92016,0.180873,0.896208,0.255629,0.952096,0.249298,0.998004,0.941617
5,ANET,183.370,1090,0.388769,0.938124,0.306084,0.964072,0.254534,0.9501,0.053264,0.886228,0.934631
6,APA,43.810,4565,0.26199,0.866267,0.183977,0.902196,0.341126,0.974052,0.164604,0.992016,0.933633
7,PSX,112.970,1770,0.343237,0.922156,0.17119,0.886228,0.214545,0.916168,0.108206,0.972056,0.924152
8,PHM,77.330,2586,0.897579,0.992016,0.467768,0.994012,0.148771,0.852295,0.022962,0.816367,0.913673
9,HES,153.280,1304,0.350265,0.924152,0.15534,0.874251,0.168551,0.88024,0.108451,0.974052,0.913174


In [120]:
writer = pd.ExcelWriter('momentum_srategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [121]:
column_formats = {
    'A':['Ticker', string_format], 
    'B':['Price', dollar_format],
    'C':['Number of Shares to Buy', integer_format],
    'D':['One-Year Price Return', percent_format],
    'E':['One-Year Return Percentile', percent_format],
    'F':['Six-Month Price Return', percent_format],
    'G':['Six-Month Return Percentile', percent_format],
    'H':['Three-Month Price Return', percent_format],
    'I':['Three-Month Return Percentile', percent_format],
    'J':['One-Month Price Return', percent_format],
    'K':['One-Month Return Percentile', percent_format],
    'L':['HQM Score', percent_format]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
writer.save()